In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('../data/chembl_33.db')

In [3]:
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
try:
    assert len(cursor.fetchall()) > 1
except AssertionError:
    print('Incorrect database. Please download the database again.')

In [4]:
_sql = """
SELECT
    MOLECULE_DICTIONARY.pref_name,
    MOLECULE_DICTIONARY.CHEMBL_ID as chembl_idx,
    ASSAYS.chembl_id as assay_id,
    ACTIVITIES.pchembl_value as pchembl_value,
    GO_CLASSIFICATION.go_id as go_id,
    GO_CLASSIFICATION.pref_name as go_term
FROM MOLECULE_DICTIONARY
JOIN ACTIVITIES ON MOLECULE_DICTIONARY.molregno == ACTIVITIES.molregno
JOIN ASSAYS ON ACTIVITIES.assay_id == ASSAYS.assay_id
JOIN TARGET_DICTIONARY on ASSAYS.tid == TARGET_DICTIONARY.tid
JOIN TARGET_COMPONENTS on TARGET_DICTIONARY.tid == TARGET_COMPONENTS.tid
JOIN COMPONENT_SEQUENCES on TARGET_COMPONENTS.component_id == COMPONENT_SEQUENCES.component_id
JOIN COMPONENT_GO on COMPONENT_SEQUENCES.component_id == COMPONENT_GO.component_id
JOIN GO_CLASSIFICATION on COMPONENT_GO.go_id == GO_CLASSIFICATION.go_id
JOIN COMPOUND_STRUCTURES on MOLECULE_DICTIONARY.molregno == COMPOUND_STRUCTURES.molregno
WHERE
    ASSAYS.tid is not null
    and ASSAYS.assay_organism = 'Homo sapiens'
"""

In [5]:
df = pd.read_sql(_sql, con=conn)
df.head()

,pref_name,chembl_idx,assay_id,pchembl_value,go_id,go_term
0,None,CHEMBL344142,CHEMBL615157,NaN,GO:0000166,nucleotide binding
1,None,CHEMBL344142,CHEMBL615157,NaN,GO:0009058,biosynthetic process
2,None,CHEMBL344142,CHEMBL615157,NaN,GO:0016491,oxidoreductase activity
3,SPRIOLACTONE,CHEMBL400534,CHEMBL615157,NaN,GO:0000166,nucleotide binding
4,SPRIOLACTONE,CHEMBL400534,CHEMBL615157,NaN,GO:0009058,biosynthetic process


In [8]:
df.dropna(subset=['pchembl_value'], inplace=True)
df.shape

(39205196, 6)

In [9]:
df.to_csv('../data/mappings/chembl_go.tsv', sep='\t', index=False)